# ai3 for the same thing

## dependencies

In [26]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

## dataset stuff

### read the dataset

In [27]:
df = pd.read_csv('dataset.csv')
df = df.sample(frac = 1)
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,diagnosis
262,60,0,1,150,240,0,0,171,0,0.9,1,0.0,3.0,0
125,45,0,2,130,234,0,2,175,0,0.6,2,0.0,3.0,0
15,57,1,3,150,168,0,0,174,0,1.6,1,0.0,3.0,0
118,63,1,4,130,330,1,2,132,1,1.8,1,3.0,7.0,3
31,60,1,4,117,230,1,0,160,1,1.4,1,2.0,7.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,41,1,4,110,172,0,2,158,0,0.0,1,0.0,7.0,1
266,52,1,4,128,204,1,0,156,1,1.0,2,0.0,?,2
233,74,0,2,120,269,0,2,121,1,0.2,1,1.0,3.0,0
111,56,1,4,125,249,1,2,144,1,1.2,2,1.0,3.0,1


### train_test split

In [28]:
ratio = 0.9 # 90:10 train:test ratio
train_set = df.sample(frac=ratio)

# Dropping all those indexes from the dataframe that exists in the train_set
test_set = df.drop(train_set.index)
train_set.shape, test_set.shape

train_set
test_set

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,diagnosis
239,42,1,2,120,295,0,0,162,0,0.0,1,0.0,3.0,0
240,41,1,2,110,235,0,0,153,0,0.0,1,0.0,3.0,0
245,67,1,4,120,237,0,0,71,0,1.0,2,0.0,3.0,2
70,65,0,3,155,269,0,0,148,0,0.8,1,0.0,3.0,0
187,66,1,2,160,246,0,0,120,1,0.0,2,3.0,6.0,2
297,57,0,4,140,241,0,0,123,1,0.2,2,0.0,7.0,1
204,43,1,4,110,211,0,0,161,0,0.0,1,0.0,7.0,0
254,43,1,4,115,303,0,0,181,0,1.2,2,0.0,3.0,0
11,56,0,2,140,294,0,2,153,0,1.3,2,0.0,3.0,0
58,54,1,3,125,273,0,2,152,0,0.5,3,1.0,3.0,0


### formating

In [31]:
# import training dataset

# Clean column names
train_set.columns = train_set.columns.str.strip()

# Handle missing values represented by '?'
train_set = train_set.replace('?', np.nan)

# Convert columns to numeric, drop rows with NaNs
train_set = train_set.dropna()
train_set = train_set.astype(float)

# Split features and labels
X = train_set.drop("diagnosis", axis=1)
y = train_set["diagnosis"]

# Convert to PyTorch tensors
tensor = torch.tensor(X.values, dtype=torch.float32)
target = torch.tensor(y.values, dtype=torch.float32)  # or LongTensor if it's class labels

print(tensor.shape, target.shape)

torch.Size([268, 13]) torch.Size([268])


## ai

### training

### testing

## ui